In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from time import sleep
from urllib.parse import quote
import pandas as pd

In [5]:
def get_href(elements):
    hrefs = []
    for elemento in elements:
        href = elemento.get_attribute('href')
        hrefs.append(href)

    return hrefs

def consultar(navegador, beneficio, start_date, end_date, municipio = 'Tanguá'):
    municipio = quote(municipio)
    navegador.get(f'https://portaldatransparencia.gov.br/beneficios/{beneficio}?de={start_date}&ate={end_date}&nomeMunicipio={municipio}&ordenarPor=nis&direcao=asc')

def consultar_Bolsa_Familia(navegador, beneficio, start_date, end_date, municipio = 'Tanguá'):
    # start_date = quote(start_date, safe='')
    # end_date = quote(end_date, safe='')
    municipio = quote(municipio)
    return f'https://portaldatransparencia.gov.br/beneficios/{beneficio}?de={start_date}&ate={end_date}&tipoBeneficio=1&nomeMunicipio={municipio}&ordenarPor=nis&direcao=asc'


In [6]:
driver = webdriver.Chrome()
consultar(driver,'bolsa-familia', '01/01/2016', '31/12/2016', 'Tanguá')
# inicia a pagian e espera 10 segundos para os dados carregarem
sleep(5)

# Clica no botão de ver paginação completa
# driver.find_element(By.XPATH, '//*[@id="lista_wrapper"]/div/div[3]/div[3]/button').click()
driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()



# Clica no select para aumentar o numero de dados na pagina
elemento_dropdown  = driver.find_element(By.XPATH, '//*[@id="lista_length"]/label/select')
Select(elemento_dropdown).select_by_value('50')

sleep(5)
# Verifica quantas paginas possuem
quantPaginas = driver.find_element(By.XPATH, '//*[@id="lista_info"]').text.split()[-1]

# Crie uma lista para armazenar os valores de atributo href
hrefs = []

# for i in range(int(quantPaginas)):
for i in range(2):
    # Busca os lnks detalhar
    elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')

    # Itere sobre cada elemento e obtenha o valor do atributo href
    hrefs.extend(get_href(elements))

    # Clica no botão proximo  
    driver.find_element(By.ID, 'lista_next').click()

    
    sleep(7)

driver.quit()

In [43]:
df = pd.DataFrame(hrefs, columns=['Detalhes_link'])

'https://portaldatransparencia.gov.br/beneficios/bolsa-familia?de=01/01/2016&ate=31/12/2016&tipoBeneficio=1&nomeMunicipio=Tangu%C3%A1&ordenarPor=nis&direcao=asc'

In [46]:
df.to_csv('2016.csv')